# Project 2: Natural Language Processing
Authors: Zechen Wu, Elena Franchini

# Investigate dataset

## Dataset selection
The dataset we will use is the "SQuAD2.0: The Stanford Question Answering Dataset". The website provides the training and validation (i.e. development) set in the form of JSON.


## Dataset analysis
Data in the training set consists of strings which represent questions and answers (that come from Wikipedia articles) and can be found as values under the 'data' key. Each 'title' key is associated to a 'paragraphs' key which is an array containing these questions and answers associated to that title (the title acts as a category). Each question is composed by the text representing the question, the id, an array of answers and a flag checking if answering to that question is impossible: if the flag is true, the array of answers is empty. In addition, each answer is associated to the 'answer_start' key whose value represent the starting position of the answer.

## Word2Vec embedding (or index documents)

# Train and evaluate model

## Train the model to perform the specific task

## Test pre-trained models on the task if they already exist

## Investigate the effectiveness of Large Language Models (LLMs) together with zero-shot and/or few-shot learning on the task

## Evaluate the different methods and compare their performance across a representative test set

# Add voice interactivity

## Investigate how effective and reliable the voice interactive components are

## If they are not particularly reliable, how might you change them to make them more robust?